In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import *

In [2]:
outputbasefolder = '../../../Algorithms/Unsupervised_Segmentation/Approaches/With_Scribble/Outputs/Human_DLPFC/Report'

In [3]:
co_ordinates = pd.read_csv('../Coordinates/coordinates.csv',index_col=0,header=0)
manual_annotation = pd.read_csv('../manual_annotations.csv',index_col=0,header=0)
pc_s = pd.read_csv('../Principal_Components/CSV/15_pcs.csv',index_col=0,header=0)
# mclust_init = pd.read_csv('../../../../Algorithms/Unsupervised_Segmentation/Approaches/With_Scribble/Outputs/Human_DLPFC/151673/many_models_1_1_2_itr_4/Outputs/Cluster_Labels_15final.csv',index_col=0,header=0)
mclust_init = pd.read_csv('components_manual_annotations.csv',index_col=0,header=0)

c2 = (co_ordinates - co_ordinates.mean())*10/ co_ordinates.std()
# c2 = co_ordinates

combined_df = pd.concat([c2,manual_annotation,mclust_init,pc_s],axis=1)
combined_df['label'].fillna(-1,inplace=True)
# combined_df = combined_df.dropna(subset=['label'])
# combined_df['label'] = combined_df['manual_annotation']
combined_df.dropna(inplace=True)
# combined_df['label'] = combined_df['label'].astype(int)
# combined_df.drop(combined_df[combined_df['label']==40].index,inplace=True)


In [5]:
label_map = {
    0: 'WM',
    6: 'L1',
    5: 'L2',
    4: 'L3',
    3: 'L4',
    2: 'L5',
    1: 'L6',
    7: 'L7',
    -1: 'undetected',
}
# combined_df['label'] = combined_df['label'].map(label_map)

In [11]:
combined_df['label'].nunique()

12

In [4]:
features = combined_df[pc_s.columns].values
features_coordinate = combined_df[['imagerow','imagecol']].values
labels = combined_df['manual_annotation'].values
features_cor = combined_df[pc_s.columns.to_list()+['imagerow','imagecol']].values
labels_ours = combined_df['label'].values

In [13]:
silhouette_cosine = silhouette_score(features,labels,metric='cosine')
silhouette_euclidean = silhouette_score(features,labels,metric='euclidean')
silhouette_cosine_cor = silhouette_score(features_cor,labels,metric='cosine')
silhouette_euclidean_cor = silhouette_score(features_cor,labels,metric='euclidean')

print('Silhouette Score for Cosine (Features only): ',silhouette_cosine)
print('Silhouette Score for Euclidean (Features only): ',silhouette_euclidean)
print('Silhouette Score for Cosine (Features + Coordinate): ',silhouette_cosine_cor)
print('Silhouette Score for Euclidean (Features + Coordinate): ',silhouette_euclidean_cor)

Silhouette Score for Cosine (Features only):  0.09140162206953495
Silhouette Score for Euclidean (Features only):  0.05522049229597379
Silhouette Score for Cosine (Features + Coordinate):  -0.026425712629437617
Silhouette Score for Euclidean (Features + Coordinate):  0.06999632733049578


In [10]:
silhouette_cosine_ours = silhouette_score(features,labels_ours,metric='cosine')
silhouette_euclidean_ours = silhouette_score(features,labels_ours,metric='euclidean')
silhouette_cosine_cor_ours = silhouette_score(features_cor,labels_ours,metric='cosine')
silhouette_euclidean_cor_ours = silhouette_score(features_cor,labels_ours,metric='euclidean')
silhouette_euclidian_cor_ours = silhouette_score(features_cor,labels_ours,metric='euclidean')

print('Silhouette Score for Cosine (Features only) (ours): ',silhouette_cosine_ours)
print('Silhouette Score for Euclidean (Features only) (ours): ',silhouette_euclidean_ours)
print('Silhouette Score for Cosine (Features + Coordinate) (ours): ',silhouette_cosine_cor_ours)
print('Silhouette Score for Euclidean (Features + Coordinate) (ours): ',silhouette_euclidean_cor_ours)

Silhouette Score for Cosine (Features only) (ours):  0.03334348166162783
Silhouette Score for Euclidean (Features only) (ours):  0.03646206473210781
Silhouette Score for Cosine (Features + Coordinate) (ours):  -0.24573542486764866
Silhouette Score for Euclidean (Features + Coordinate) (ours):  0.00820058255202031


In [21]:
def distance(cos_sim,euclidean_dist,alpha=0.9):
    return alpha * cos_sim + (1-alpha) * euclidean_dist

euclidean_dist_c = euclidean_distances(features_coordinate,features_coordinate)
cos_sim_dist_c = cosine_similarity(features,features) + 1

custom_dist_c = distance(cos_sim_dist_c,euclidean_dist_c,alpha=0.7)
np.fill_diagonal(custom_dist_c,0)
silhouette_custom = silhouette_score(custom_dist_c,labels,metric='precomputed')
print('Silhouette Score for Custom: ',silhouette_custom)

Silhouette Score for Custom:  -0.003147402318972365


In [73]:
silhouette_custom_manual = silhouette_score(custom_dist_c,labels_ours,metric='precomputed')
print('Silhouette Score for Custom Manual: ',silhouette_custom_manual)

Silhouette Score for Custom Manual:  -0.04742275631833217


In [56]:
sample2 = pd.read_csv('../../../Algorithms/Unsupervised_Segmentation/Approaches/With_Scribble/Outputs/Human_DLPFC/151673/many_models_1_1_2_itr_2/Outputs/Cluster_Labels_15final.csv',index_col=0,header=0)

labels_ours = pd.concat([combined_df.drop(columns=['label']),sample2],axis=1,join='inner').fillna(-1)['label'].values

silhouette_cosine_ours = silhouette_score(features,labels_ours,metric='cosine')
silhouette_euclidean_ours = silhouette_score(features_coordinate,labels_ours,metric='euclidean')
silhouette_custom_ours = silhouette_score(custom_dist_c,labels_ours,metric='precomputed')
print('Silhouette Score for Cosine ours: ',silhouette_cosine_ours)
print('Silhouette Score for Euclidean ours: ',silhouette_euclidean_ours)
print('Silhouette Score for Custom ours: ',silhouette_custom_ours)


Silhouette Score for Cosine ours:  0.04709180024361259
Silhouette Score for Euclidean ours:  -0.04493280891503934
Silhouette Score for Custom ours:  -0.054673875134121745


### Calinski and Harabasz score

In [12]:
Calinski_Harabaz_score = calinski_harabasz_score(features,labels)
print('Calinski Harabaz Score: ',Calinski_Harabaz_score)
calinski_harabasz_score_features_coordinates = calinski_harabasz_score(features_cor,labels)
print('Calinski Harabaz Score for features coordinates: ',calinski_harabasz_score_features_coordinates)
calinski_harabasz_score_ours = calinski_harabasz_score(features,labels_ours)
print('Calinski Harabaz Score ours: ',calinski_harabasz_score_ours)
calinski_harabasz_score_features_coordinates_ours = calinski_harabasz_score(features_cor,labels_ours)
print('Calinski Harabaz Score for features coordinates ours: ',calinski_harabasz_score_features_coordinates_ours)


Calinski Harabaz Score:  867.3427564955067
Calinski Harabaz Score for features coordinates:  533.2242890984767
Calinski Harabaz Score ours:  484.1004858776906
Calinski Harabaz Score for features coordinates ours:  353.7853000089061
